### Метод BFGS

#### Условие Вольфе
Сперва реализуем линейный поиск с условиями Вольфе. Это нам понадобится при  реализации BFGS (он опирается на одномерный поиск, для которого соблюдение условий Вольфа обязательно).

In [1]:
from matplotlib import pyplot as plt
import numpy as np
plt.rcParams["figure.figsize"] = (20,10)

In [ ]:
def wolfe_search(f, x, p, grad, nabla):
    '''
    Поиск с условиями Вольфе
    '''
    a = 1
    c1 = 1e-4 
    c2 = 0.9 
    fx = f(x)
    x_new = x + a * p 
    nabla_new = grad(f, x_new)
    while f(x_new) >= fx + (c1*a*nabla.T @ p) or nabla_new.T @ p <= c2*nabla.T @ p : 
        a *= 0.5
        x_new = x + a * p 
        nabla_new = grad(f, x_new)
    return a

#### Реализация метода

Что нужно иметь в виду: 
1. в алгоритме не используется и не вычисляется Гессиан как таковой, нам нужно только
хорошее приближение.
2. по этой причине асимптотика у каждого шага O(n^2), а не O(n^3), это важное преимущество над обычным метоодом Ньютона
3. BFGS более общий, чем метод Гаусса-Ньютона.

In [ ]:
def BFGS(f,x0, epochs, grad):
    '''
    Реализация BFGS
    Параметры
    f:      целевая функция 
    x0:     начальная гипотеза
    max_it: максимальное число итераций 
    plot:   if the problem is 2 dimensional, returns 
            a trajectory plot of the optimisation scheme.
    OUTPUTS: 
    x:      the optimal solution of the function f 
    '''
    d = len(x0) # наша размерность 
    nabla = grad(f,x0) # градиент в начальной точке
    I = np.eye(d) # единичная матрица

    H = I.copy() # начальный обратный гессиан
    x = x0[:]
    it = 2 

    while np.linalg.norm(nabla) > 1e-5 and (it < epochs): # while gradient is positive
        it += 1
        p = -H @ nabla # направление поиска
        a = wolfe_search(f, x, p, nabla) # поиск с условиями Вольфе 
        s = np.array([a * p]) # величина шага
        x_new = x + a * p 
        nabla_new = grad(f, x_new)
        y = np.array([nabla_new - nabla]) 
        y, s = np.reshape(y,(d,1)), np.reshape(s,(d,1))
        y_trans, s_trans = y.transpose(), s.transpose()
        
        r = 1/(y_trans @ s)

        
        li = (I-(r*(s @ (y_trans))))
        ri = (I-(r*(y @ (s_trans))))
        hess_inter = li @ H @ ri
        H = hess_inter + (r*(s @ (s_trans))) # обновление (обратного) гессиана
        
        nabla = nabla_new.copy()
        x = x_new.copy()
        
        points = np.append(points,[x], axis=0) # 
    return points
